This file contains the code to find the most suitable K value for the KNN filling algorithm
Since Logistic regression performs and worst among all algorithm, we don't use it here

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.csvcsv)
import sklearn
# Cross Validation Classification Accuracy
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from sklearn import preprocessing 
from sklearn.impute import KNNImputer
#import matplotlib.pyplot as plot
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score
from sklearn import tree
import statistics
import time
import warnings
warnings.filterwarnings('ignore')
# plotting errorbar graph for visual representation of the performance of the classifiers
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
import random
from xgboost import XGBClassifier
import shutil
import os
from our_evaluate_sepsis_score import evaluate_performance
import our_functions_library as flib
# import numba
# from numba import jit, njit, cuda

In [2]:
# Global initialization, get all the data from file and generate all the needed variable
filename = '../../raw_data/raw_data_400.csv' # use raw dataset
originalData = pd.read_csv(filename) # read csv data into DataFrame var raw
print("Data size:",originalData.shape)
Uniq_ID= np.unique(originalData['Patient_id']) 
print('Patient id size:',len(Uniq_ID))
print(Uniq_ID)
Original_Uniq_ID_head = range(1,21)
print("length of an array",len(Original_Uniq_ID_head))
print(Original_Uniq_ID_head)

X_columns = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 
             'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
             'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 
             'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2','HospAdmTime',
             'ICULOS', 'Patient_id', 'time']
y_columns = ['Patient_id', 'SepsisLabel']

# To test if these 3 columns is useful
# originalData = originalData.drop(['Unit1', 'Unit2', 'EtCO2'], axis=1)
# X_columns.remove('Unit1')
# X_columns.remove( 'Unit2')
# X_columns.remove( 'EtCO2')

# Initialize the empty array X_train, X_test, y_train, y_test
X_train = pd.DataFrame(columns = X_columns)
X_test = pd.DataFrame(columns = X_columns)
y_train = pd.DataFrame(columns = y_columns)
y_test = pd.DataFrame(columns = y_columns)

# Below are the lists for KNN results of different K value
KNN_UtilityScore_mean = []
KNN_UtilityScore_std = []
KNN_F1Score_mean = []
KNN_F1Score_std = []
KNN_auroc_mean = []
KNN_auprc_mean = []
KNN_accuracy_mean = []
KNN_accuracy_std = []
KNN_positiveprediction_mean = []
KNN_baseline_mean = [ ]
KNN_total_Time= [ ]
fillmethod =""  

Data size: (15348, 43)
Patient id size: 400
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 23

In [3]:
#This block will shuffle the id sets of patients in a fix manner, so for every time you run the code, the dataset and trainset are always the same.
seed = 2
random.seed(seed)
if(np.array_equal(Original_Uniq_ID_head,Uniq_ID[:20]) ):
    print("The first 20 patient ids are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.")   
    random.shuffle(Uniq_ID)# randomly sorted the patient IDs    
else:
    print("They are not the same. Uniq_id has already been shuffled.")    
print("The original uniq id set is:\n",Uniq_ID[0:20])
print("The fixed shuffelld id set, it should be 211 275 153 189 184 110 124  49...\n",Uniq_ID[:20])

The first 20 patient ids are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.
The original uniq id set is:
 [211 275 153 189 184 110 124  49 345  73 284  20  71 328 136 241 171 141
 329 204]
The fixed shuffelld id set, it should be 211 275 153 189 184 110 124  49...
 [211 275 153 189 184 110 124  49 345  73 284  20  71 328 136 241 171 141
 329 204]


In [4]:
def KNN_reset():
    #KNN with different K size, 
    KNN_UtilityScore_mean.clear()
    KNN_UtilityScore_std.clear()
    KNN_F1Score_mean.clear()
    KNN_F1Score_std.clear()
    KNN_auroc_mean.clear()
    KNN_auprc_mean.clear()
    KNN_accuracy_mean.clear()
    KNN_accuracy_std.clear()
    KNN_positiveprediction_mean.clear()
    KNN_baseline_mean.clear()
    KNN_total_Time.clear()
    
def generateTrainDataSet(test_patienIds, X_train, y_train):
    #print("test_patienIds: \n", test_patienIds)
    train_data= originalData[~(originalData.Patient_id.isin(test_patienIds))]
    #X_train = originalData[originalData['Patient_id'].isin(patienIds)]
    #print("Xtrain after isin operation\n", train_data)
    #X_train=originalData.loc[originalData['Patient_id'] == patienIds]
    X_train = train_data[X_columns]
    #print("x_train sliced\n", X_train)
    y_train = train_data[y_columns]
    #print("y_train sliced\n", y_train)    
    return X_train, y_train
 
def generateTestDataSet(patienIds, X_test, y_test):
    print("test_patienIds: \n", patienIds)
    test_data = originalData[originalData['Patient_id'].isin(patienIds)]
    X_test = test_data[X_columns]
    #print("x_test sliced\n", X_test)
    y_test = test_data[y_columns]
    #print("y_test sliced\n", y_test)
    return X_test, y_test

# def CalculateSOFAScore(X_train_impute, X_test_impute):
#     pass

In [5]:
# Code for K-fold algorithm 
def KFold_patient(model, KforKFold=10,KforKNN=5, fillmethod=""):
    start = time.time()# time indicator for how long the Kfold func takes
    global X_train, X_test, y_train, y_test
    print("Datasize",len(Uniq_ID))
    print("K Fold of ",KforKFold ," folds with KNN =",KforKNN)
    
    # initialisation of the array for storing the different intermediate results
    accuracy_model = []
    F1Score_model = []
    baseline_model= []
    auroc_model = []
    auprc_model = []
    physio_accuracy_model = []
    f_measure_model = []
    utility_score_model = []
    mean_train = 0
    positivepredictions = []
    #The unique id sets have been created and shuffled in a fix manner in the third block, you can just use it here and no more any other manipulation
    idSets = np.array_split(Uniq_ID, KforKFold)# divide the ids into K groups
    #print("Patients number: ", len(Uniq_ID)) #print('idSets arrays',idSets)
    #This for loop is for Kfold, calculating the results for K times
    
    Twrite = time.time()
    open("test.txt", "w").close() # clear contents of existing file
    for i in range(KforKFold):
        splitted = " ".join( repr(e) for e in idSets[i])
        file1 = open("test.txt","a")
        file1.write("\n\n")
        file1.write(str("[" +splitted+"]"))
        file1.write("\n\n")
        file1.close()
    print("Time for writing id to file:", round(time.time()- Twrite,3))
    
    
    for i in range(KforKFold):
        Uniq_ID_copy= Uniq_ID.copy()
        start1 = time.time()
        X_train, X_test, y_train, y_test = flib.clearAllDatasets(X_train, X_test, y_train, y_test) #first clear all the datasets
        print("for the",i+1,"th iteration: ")#,idSets[i])   
        t1 = time.time()
        X_test,y_test = generateTestDataSet(idSets[i], X_test, y_test)
        print("Time for splitting id to test:", round(time.time()- t1,3))
        
        t2= time.time()
        id_sort= Uniq_ID_copy.sort(axis=0)
#         print("id_sort", Uniq_ID_copy)
       # print("train id", np.delete(Uniq_ID_copy, idSets[i]-1))
        X_train, y_train = generateTrainDataSet(idSets[i],X_train, y_train)
        print("Time for splitting id to train:", round(time.time()- t2,3))

        #printDataset()
        #Now the train and test dataset is generated
        #we can begin to train the model wit the training set and evaaulate the performance with the test sett   
#         X_train = X_train.drop('time', 1) X_test = X_test.drop('time', 1)
        X_train=X_train.astype('float32')
        X_test=X_test.astype('float32')    
        YTest_copy = y_test  # variable of joining the filled data (X) and Y (Train_output) 
        patientID_ytest = y_test['Patient_id']
        y_train = y_train.drop('Patient_id', 1)
        y_test = y_test.drop('Patient_id', 1)      
        y_train=y_train.astype('float64')
        y_test=y_test.astype('float64')
        #print('YTest',YTest_copy.head())
        
        #fill the missing data
        if X_train.isnull().values.any() or X_test.isnull().values.any() :
            tKNN= time.time()
            print("X_train or X_test contains NaN values, KNN/mean is performed.")
            #if there is missing value
            #
            X_train_impute, X_test_impute, fillmethod = flib.KNNfilling(X_train, X_test, KforKNN, fillmethod)
            #X_train_impute, X_test_impute,fillmethod = flib.MeanFilling(X_train,X_test, fillmethod)
            #check the missing data   
            if np.isnan(X_train_impute).any() or np.isnan(X_test_impute).any() :
                print("X_train_impute or X_test_impute still contains NaN values")        
            else:
                print("X_train_impute or X_test_impute have all been filled ")
            print("Time for NaN values filling: ", round(time.time()- tKNN,2) )
        else:
            print("X_train or X_test have all been filled: ")
            X_train_impute = X_train
            X_test_impute = X_test
                 
        #Scale the data， uncomment this part for logistic regression only, in other case, put them in comment
        model_name = type(model).__name__
        if (model_name == 'LogisticRegression'):
            print("scaled!")
            scaler = preprocessing.StandardScaler()
            scaler.fit(X_train_impute)
            X_train_impute = scaler.transform(X_train_impute)
            X_test_impute = scaler.transform(X_test_impute)

        #fit the model and predict
        model.fit(X_train_impute, y_train)
        y_predicted = model.predict(X_test_impute).astype(int)   
        y_predicted_probobility = model.predict_proba(X_test_impute) 
        #transfer the outpu and evalute it
#         y_labels = y_test.astype(int).to_numpy()
        y_labels = y_test.astype(int).to_numpy()
        if(len(y_predicted_probobility[0]) ==2 ):
            y_predicted_probobility =  y_predicted_probobility[:,1].round(4)
        else:
            print("columns of y_predicted_probobility is: ",len(y_predicted_probobility[0]))
            y_predicted_probobility =  y_predicted_probobility.round(4)
        #print(y_labels) 
        auroc, auprc, physio_accuracy, f_measure, utility_score = evaluate_performance(y_labels, y_predicted, y_predicted_probobility,patientID_ytest) 
        auroc_model.append(round(auroc,4))
        auprc_model.append(round(auprc,4))
        physio_accuracy_model.append(round(physio_accuracy,4))
        f_measure_model.append(round(f_measure,4))
        utility_score_model.append(round(utility_score,4))
        positivepredictions.append(np.sum(y_predicted))        
        baseline_model.append(round( (1 - float(y_test.mean()) )*100 , 2 ))
        print("\ny_test size:",y_test.shape, '1´s in y_test',y_test.sum())
        print("The number of 1 (SepsisLabel) in this prediction: ", np.sum(y_predicted))
        print("Time spent in this KFold iteration",round( time.time()-start1,2),"sec.\n")
        print("******************************************************************")    
    print("\nEvaluation parameters of the utiltiy evaluation function:")
    print('auroc of model:', auroc_model)
    print('auprc of model:', auprc_model)
    print('utility F1 of model:', f_measure_model)
    print('Utility accuracy of model:', accuracy_model) 
    print('Utility score of model:', utility_score_model)
    print("Positive Prediction: ", positivepredictions)
    print('Baseline model:', baseline_model)  
    KNN_auroc_mean.append(np.mean(auroc_model))
    KNN_auprc_mean.append(np.mean(auprc_model))
    KNN_F1Score_mean.append(np.mean(f_measure_model))
    KNN_F1Score_std.append(np.std(f_measure_model))
    KNN_accuracy_mean.append(np.mean(physio_accuracy_model))
    KNN_accuracy_std.append(np.std(physio_accuracy_model))
    KNN_UtilityScore_mean.append(np.mean(utility_score_model))
    KNN_UtilityScore_std.append(np.std(utility_score_model))
    KNN_positiveprediction_mean.append(np.mean(positivepredictions))
    KNN_baseline_mean.append(np.mean(baseline_model))
    
    totalTime=round(time.time()- start,2)
    KNN_total_Time.append(totalTime)
    print("\nTotal Time spent in  KFold function",totalTime,"sec.\n")

In [6]:
def Print_DATA():
    print("Numbers of K:",len(KNN_accuracy_mean))
    print("KNN_UtilityScore_mean", KNN_UtilityScore_mean)
    print("KNN_UtilityScore_std", KNN_UtilityScore_std)
    print("KNN_F1Score_mean", KNN_F1Score_mean)
    print("KNN_F1Score_std", KNN_F1Score_std)
    print("KNN_auroc_mean", KNN_auroc_mean)
    print("KNN_auprc_mean", KNN_auprc_mean)
    print("KNN_accuracy_mean", KNN_accuracy_mean)
    print("KNN_accuracy_std", KNN_accuracy_std)
    print("KNN_positiveprediction_mean", KNN_positiveprediction_mean)
    print("KNN_baseline_mean", KNN_baseline_mean)
    print("KNN_total_time", KNN_total_Time)      

# def findBestKforKNN(model, KforKFold=10,KforKNNstart=1,KforKNNend=10, stepsize=5):
#     if(KforKNNend<=1):
#         print("K must be a interger larger than 1")
#         return
#     KNN_reset()
#     step=stepsize
#     for i in range(KforKNNstart,KforKNNend+1,step) : 
#         print("KNN of K = ",i)    
#         KFold_patient(model,10,i)

In [7]:
#logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='saga', max_iter=1000,penalty='l1')
logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='lbfgs', max_iter=1000)
decisionTreeModel = tree.DecisionTreeClassifier(random_state=2)
randomForestModel = RandomForestClassifier(random_state=2)
XGBoostModel = XGBClassifier(random_state=2, n_jobs=4)
#AdaBoostModel = AdaBoostClassifier(random_state=2)

In [8]:
#fill the missing data with one function
print(filename)
#print(originalData)
if (originalData.isnull().values.any()):
    print('Data isn´t all filled before K-Fold Func')
#data,fillmethod = flib.linearFillingAll(originalData,1, True)#forwardfilling + fill other missing data with 0


../../raw_data/raw_data_400.csv
Data isn´t all filled before K-Fold Func


In [ ]:
KFold_patient(decisionTreeModel,10,5)

Datasize 400
K Fold of  10  folds with KNN = 5
Time for writing id to file: 0.003
for the 1 th iteration: 
test_patienIds: 
 [211 275 153 189 184 110 124  49 345  73 284  20  71 328 136 241 171 141
 329 204 157 290 305 281 323  57 208  41 366 220 120 118 244  22 325 222
 178 369 381  52]
Time for splitting id to test: 0.003
Time for splitting id to train: 0.005
X_train or X_test contains NaN values, KNN/mean is performed.
X_train_impute or X_test_impute have all been filled 
Time for NaN values filling:  28.09

y_test size: (1535, 1) 1´s in y_test SepsisLabel    48.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  29
Time spent in this KFold iteration 28.41 sec.

******************************************************************
for the 2 th iteration: 
test_patienIds: 
 [ 50 379 363 326 248 348 169 319 149 210 113  77 352 342 292 324 309 285
 310  32 320 332 331 144 223 151  69 216 321 276 102 249 283  46  38 245
 376  61 289 146]
Time for splitting id to test: 0.002

In [ ]:
displayCurrentResult(1,9)

In [ ]:
df1 = da.